In [15]:
import arcpy
arcpy.env.overwriteOutput = True
project_loc = "F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Archive\ForestHealth_ThresholdData.gdb\\"

# Input parameters
clip_feature = r"F:\GIS\PARCELUPDATE\Workspace\Vector.sde\SDE.TRPA_bdy"
output_feature = project_loc + "Caldor_RVG_ClippedTRPA"
input_feature = project_loc + "Caldor_RAVG"

output_stats = project_loc + "Caldor_RAVG_gridcode_summary"
output_treatment_stats = project_loc + "Caldor_RAVG_withtreatment_gridcode_summary"
fuel_treatment_fc = project_loc + "Tahoe_Forest_Fuels_Tx_ExportFeatures_Pre2022"
caldor_ravg_treatment_clipped = project_loc + "Caldor_RAVG_Clipped_WithTreatment"
treatment_selection = project_loc + "FuelsTx_Pre2022"
    

try:
    # Check if the clipped veg severity already exists
    if arcpy.Exists(output_feature):
        print("Clipped layer exists")
    else:
        arcpy.analysis.Clip(in_features=input_feature, 
                        clip_features=clip_feature, 
                        out_feature_class=output_feature)

        print(f"Clip completed successfully. Output saved to: {output_feature}")
    
    # Summary Statistics: Sum the SHAPE_AREA grouped by GRIDCODE
    arcpy.Statistics_analysis(
        in_table=output_feature, 
        out_table=output_stats, 
        statistics_fields=[["SHAPE_AREA", "SUM"]], 
        case_field="GRIDCODE"
    )
    print(f"Summary statistics saved to: {output_stats}")
     # Calculate the total shape_area for all gridcodes
    total_area = 0
    with arcpy.da.SearchCursor(output_stats, ["SUM_SHAPE_AREA"]) as cursor:
        for row in cursor:
            total_area += row[0]

    print(f"Total Shape Area across all gridcodes: {total_area}")
    
    # Add a new field for the percentage
    arcpy.AddField_management(output_stats, "PERCENTAGE", "DOUBLE")
    
    # Update the percentage field for each gridcode
    with arcpy.da.UpdateCursor(output_stats, ["GRIDCODE", "SUM_SHAPE_AREA", "PERCENTAGE"]) as cursor:
        for row in cursor:
            percentage = (row[1] / total_area) * 100
            row[2] = percentage
            cursor.updateRow(row)

    print("Percentage field updated successfully.")
    
    treatment_selection = "F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb\Tahoe_Forest_Fuels_Tx_ExportFeatures_Pre2022"
    #arcpy.analysis.Select(in_features=fuel_treatment_fc, out_feature_class=treatment_selection, where_clause="YEAR < 2022")    
    
    arcpy.analysis.Clip(in_features=output_feature, clip_features=treatment_selection, out_feature_class=caldor_ravg_treatment_clipped)
    
    arcpy.Statistics_analysis(
        in_table=caldor_ravg_treatment_clipped, 
        out_table=output_treatment_stats, 
        statistics_fields=[["SHAPE_AREA", "SUM"]], 
        case_field="GRIDCODE"
    )
    print(f"Summary of treatment statistics saved to: {output_treatment_stats}")
     
    # Calculate the total shape_area for all gridcodes
    total_area = 0
    with arcpy.da.SearchCursor(output_treatment_stats, ["SUM_SHAPE_AREA"]) as cursor:
        for row in cursor:
            total_area += row[0]

    print(f"Total Shape Area across all gridcodes in treatment areas: {total_area}")
    
     # Add a new field for the percentage
    arcpy.AddField_management(output_treatment_stats, "PERCENTAGE", "DOUBLE")
    
    # Update the percentage field for each gridcode
    with arcpy.da.UpdateCursor(output_treatment_stats, ["GRIDCODE", "SUM_SHAPE_AREA", "PERCENTAGE"]) as cursor:
        for row in cursor:
            percentage = (row[1] / total_area) * 100
            row[2] = percentage
            cursor.updateRow(row)
    
except arcpy.ExecuteError:
    print(f"Error occurred: {arcpy.GetMessages(2)}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

Clipped layer exists
Summary statistics saved to: F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Archive\ForestHealth_ThresholdData.gdb\Caldor_RAVG_gridcode_summary
Total Shape Area across all gridcodes: 40141285.07400812
Percentage field updated successfully.
Summary of treatment statistics saved to: F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Archive\ForestHealth_ThresholdData.gdb\Caldor_RAVG_withtreatment_gridcode_summary
Total Shape Area across all gridcodes in treatment areas: 39973299.911060676


In [2]:
import arcpy
arcpy.env.overwriteOutput = True
project_loc = "F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Archive\ForestHealth_ThresholdData.gdb\\"

caldor_bndy = project_loc + "Caldor_RVG_ClippedTRPA"
project_loc = "F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Archive\ForestHealth_ThresholdData.gdb\\"
fuel_treatment_fc = project_loc + "Tahoe_Forest_Fules_2010to2022"
caldor_ravg_treatment_clipped = project_loc + "Caldor_RAVG_Clipped_WithTreatment"
output_treatment_stats = project_loc + "Caldor_RAVG_withtreatment_gridcode_summary"

arcpy.analysis.Clip(in_features=caldor_bndy, 
                    clip_features=fuel_treatment_fc, 
                    out_feature_class=caldor_ravg_treatment_clipped)

print(f"Clip completed successfully. Output saved to: {caldor_ravg_treatment_clipped}")
    
arcpy.Statistics_analysis(
    in_table=caldor_ravg_treatment_clipped, 
    out_table=output_treatment_stats, 
    statistics_fields=[["SHAPE_AREA", "SUM"]], 
    case_field="GRIDCODE"
)
print(f"Summary of treatment statistics saved to: {output_treatment_stats}")

# Calculate the total shape_area for all gridcodes
total_area = 0
with arcpy.da.SearchCursor(output_treatment_stats, ["SUM_SHAPE_AREA"]) as cursor:
    for row in cursor:
        total_area += row[0]

print(f"Total Shape Area across all gridcodes in treatment areas: {total_area}")

 # Add a new field for the percentage
arcpy.AddField_management(output_treatment_stats, "PERCENTAGE", "DOUBLE")

# Update the percentage field for each gridcode
with arcpy.da.UpdateCursor(output_treatment_stats, ["GRIDCODE", "SUM_SHAPE_AREA", "PERCENTAGE"]) as cursor:
    for row in cursor:
        percentage = (row[1] / total_area) * 100
        row[2] = percentage
        cursor.updateRow(row)
print("done")

Clip completed successfully. Output saved to: F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Archive\ForestHealth_ThresholdData.gdb\Caldor_RAVG_Clipped_WithTreatment
Summary of treatment statistics saved to: F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Archive\ForestHealth_ThresholdData.gdb\Caldor_RAVG_withtreatment_gridcode_summary
Total Shape Area across all gridcodes in treatment areas: 5229469.685183449
done
